In [1]:
import numpy as np 
import pandas as pd 
import time

pd.set_option('mode.chained_assignment', None)

In [2]:
def diff_KDA(df):
    df['team1_KDA_0']=df[["player1_kills", "player2_kills","player3_kills","player4_kills","player5_kills",
                      "player1_assists", "player2_assists","player3_assists","player4_assists","player5_assists"]].sum(axis=1)
    df['team1_KDA_1']=df[["player1_deaths", "player2_deaths","player3_deaths","player4_deaths","player5_deaths"]].sum(axis=1)
    df['team1_KDA']=df['team1_KDA_0']/df['team1_KDA_1']
    df['team2_KDA_0']=df[["player6_kills", "player7_kills","player8_kills","player9_kills","player10_kills",
                          "player6_assists", "player7_assists","player8_assists","player9_assists","player10_assists"]].sum(axis=1)
    df['team2_KDA_1']=df[["player6_deaths", "player7_deaths","player8_deaths","player9_deaths","player10_deaths"]].sum(axis=1)
    df['team2_KDA']=df['team2_KDA_0']/df['team2_KDA_1']
    df['diff_KDA']=df['team1_KDA']-df['team2_KDA']
    return df   

In [3]:
#预处理
def gettrain(df):
    df['team1_avg_kills']=df[["player1_kills", "player2_kills","player3_kills","player4_kills","player5_kills"]].mean(axis=1)
    df['team2_avg_kills']=df[["player6_kills", "player7_kills","player8_kills","player9_kills","player10_kills"]].mean(axis=1)
    df['diff_kills']=(df['team1_avg_kills']-df['team2_avg_kills'])
    
    df['team1_avg_assists']=df[["player1_assists", "player2_assists","player3_assists","player4_assists","player5_assists"]].mean(axis=1)
    df['team2_avg_assists']=df[["player6_assists", "player7_assists","player8_assists","player9_assists","player10_assists"]].mean(axis=1)
    df['diff_assists']=(df['team1_avg_assists']-df['team2_avg_assists'])
    
    df['team1_avg_deaths']=df[["player1_deaths", "player2_deaths","player3_deaths","player4_deaths","player5_deaths"]].mean(axis=1)
    df['team2_avg_deaths']=df[["player6_deaths", "player7_deaths","player8_deaths","player9_deaths","player10_deaths"]].mean(axis=1)
    df['diff_deaths']=(df['team1_avg_deaths']-df['team2_avg_deaths'])
    
    df['team1_sum_gold']=df[["player1_goldEarned", "player2_goldEarned","player3_goldEarned","player4_goldEarned","player5_goldEarned"]].sum(axis=1)
    df['team2_sum_gold']=df[["player6_goldEarned", "player7_goldEarned","player8_goldEarned","player9_goldEarned","player10_goldEarned"]].sum(axis=1)
    df['diff_gold']=df['team1_sum_gold']-df['team2_sum_gold']
    
    df=diff_KDA(df)
    return df

In [14]:
#生成特征表
def getfeature(train,df1):
    tra = train.copy()
    df = df1.copy()
    f = pd.DataFrame()
    
    f['index']=df['index']
    
    f['win']=df['team1_win']
    
    f['blood']=df['team1_firstBlood'].astype('int') *(1/600)

    f['tower']=df['team1_firstTower'].astype('int') *(1/600)

    f['inhibitor']=df['team1_firstInhibitor'].astype('int') *(2/150)

    f['baron']=df['team1_firstBaron'].astype('int') *(1/600)

    f['dragon']=df['team1_firstDragon'].astype('int') *(1/300)

    f['riftHerald']=df['team1_firstRiftHerald'].astype('int')*(1/300)

    #同时一血一龙
    df['firstBlood&firstDragon']=0.5
    x = df[(df['team1_firstBlood'] == 1) & (df['team1_firstDragon'] == 1)].copy()
    x.loc[:,'firstBlood&firstDragon']=1
    df[(df['team1_firstBlood'] == 1) & (df['team1_firstDragon'] == 1)]=x
    x = df[(df['team1_firstBlood'] == 0) & (df['team1_firstDragon'] == 0)].copy()
    x.loc[:,'firstBlood&firstDragon']=0
    df[(df['team1_firstBlood'] == 0) & (df['team1_firstDragon'] == 0)]=x
    f['blood&dragon']=df['firstBlood&firstDragon'] *(1/300)
    f['blood&dragon'] = f['blood&dragon']
    
    #同时一龙一先锋
    df['firstRiftHerald&firstDragon']=0.5
    x = df[(df['team1_firstRiftHerald'] == 1) & (df['team1_firstDragon'] == 1)].copy()
    x.loc[:,'firstRiftHerald&firstDragon']=1
    df[(df['team1_firstRiftHerald'] == 1) & (df['team1_firstDragon'] == 1)]=x
    x = df[(df['team1_firstRiftHerald'] == 0) & (df['team1_firstDragon'] == 0)].copy()
    x.loc[:,'firstRiftHerald&firstDragon']=0
    df[(df['team1_firstRiftHerald'] == 0) & (df['team1_firstDragon'] == 0)]=x
    f['dragon&riftHerald']=df['firstRiftHerald&firstDragon'] *(1/600)
    f['dragon&riftHerald'] = f['dragon&riftHerald']
    
    #同时一男爵一先锋
    df['firstBaron&firstRiftHerald']=0.5
    x = df[(df['team1_firstBaron'] == 1) & (df['team1_firstRiftHerald'] == 1)].copy()
    x.loc[:,'firstBaron&firstRiftHerald']=1
    df[(df['team1_firstBaron'] == 1) & (df['team1_firstRiftHerald'] == 1)]=x
    x = df[(df['team1_firstBaron'] == 0) & (df['team1_firstRiftHerald'] == 0)].copy()
    x.loc[:,'firstBaron&firstRiftHerald']=0
    df[(df['team1_firstBaron'] == 0) & (df['team1_firstRiftHerald'] == 0)]=x
    f['baron&riftHerald']=df['firstBaron&firstRiftHerald'] *(1/150)
    f['baron&riftHerald'] = f['baron&riftHerald']
    
    #同时一塔一水晶
    df['firstTower&firstInhibitor']=0.5
    x = df[(df['team1_firstTower'] == 1) & (df['team1_firstInhibitor'] == 1)].copy()
    x.loc[:,'firstTower&firstInhibitor']=1
    df[(df['team1_firstTower'] == 1) & (df['team1_firstInhibitor'] == 1)]=x
    x = df[(df['team1_firstTower'] == 0) & (df['team1_firstInhibitor'] == 0)].copy()
    x.loc[:,'firstTower&firstInhibitor']=0
    df[(df['team1_firstTower'] == 0) & (df['team1_firstInhibitor'] == 0)]=x
    f['tower&inhibitor']=df['firstTower&firstInhibitor'] *(2/150)
    f['tower&inhibitor'] = f['tower&inhibitor']
    
    #击杀差
    df['team1_kills']=df[["player1_kills", "player2_kills","player3_kills","player4_kills","player5_kills"]].mean(axis=1)
    df['team2_kills']=df[["player6_kills", "player7_kills","player8_kills","player9_kills","player10_kills"]].mean(axis=1)
    df['diff_kills']=(df['team1_kills']-df['team2_kills'])
    df['diff_kills']=(df['diff_kills']-min(tra['diff_kills']))/(max(tra['diff_kills'])-min(tra['diff_kills']))
    f['diff_kills']=df['diff_kills'] *(2/15)
    f['diff_kills'] = f['diff_kills']
    
    #助攻差
    df['team1_assists']=df[["player1_assists", "player2_assists","player3_assists","player4_assists","player5_assists"]].mean(axis=1)
    df['team2_assists']=df[["player6_assists", "player7_assists","player8_assists","player9_assists","player10_assists"]].mean(axis=1)
    df['diff_assists']=(df['team1_assists']-df['team2_assists'])
    df['diff_assists']=(df['diff_assists']-min(tra['diff_assists']))/(max(tra['diff_assists'])-min(tra['diff_assists']))
    f['diff_assists']=df['diff_assists'] *(1/15)
    f['diff_assists'] = f['diff_assists']
    
    #死亡差
    df['team1_deaths']=df[["player1_deaths", "player2_deaths","player3_deaths","player4_deaths","player5_deaths"]].mean(axis=1)
    df['team2_deaths']=df[["player6_deaths", "player7_deaths","player8_deaths","player9_deaths","player10_deaths"]].mean(axis=1)
    df['diff_deaths']=(df['team1_deaths']-df['team2_deaths'])
    df['diff_deaths']=(df['diff_deaths']-min(tra['diff_deaths']))/(max(tra['diff_deaths'])-min(tra['diff_deaths']))
    f['diff_deaths']=df['diff_deaths'] *(1/15)
    f['diff_deaths'] = f['diff_deaths']
    
    #经济差
    df['team1_sum_gold']=df[["player1_goldEarned", "player2_goldEarned","player3_goldEarned","player4_goldEarned","player5_goldEarned"]].sum(axis=1)
    df['team2_sum_gold']=df[["player6_goldEarned", "player7_goldEarned","player8_goldEarned","player9_goldEarned","player10_goldEarned"]].sum(axis=1)
    df['diff_gold']=df['team1_sum_gold']-df['team2_sum_gold']
    df['diff_gold']=(df['diff_gold']-min(tra['diff_gold']))/(max(tra['diff_gold'])-min(tra['diff_gold']))
    f['diff_gold']=df['diff_gold'] *(40/30)
    f['diff_gold'] = f['diff_gold']
    
    #KDA差
    df=diff_KDA(df)
    df['diff_KDA']=(df['diff_KDA']-min(tra['diff_KDA']))/(max(tra['diff_KDA'])-min(tra['diff_KDA']))
    f['diff_KDA']=df['diff_KDA'] *(12/15)
    f['diff_KDA'] = f['diff_KDA']
    
    f['sum'] = f['blood'] + f['tower'] + f['inhibitor'] + f['baron'] + f['dragon'] + f['riftHerald'] + f['blood&dragon'] \
            + f['dragon&riftHerald'] + f['baron&riftHerald'] + f['tower&inhibitor'] + f['diff_kills'] + f['diff_assists']\
            + f['diff_deaths'] + f['diff_gold'] + f['diff_KDA']
     
    #f['sum'] = f['blood'].map(str) + f['tower'].map(str) + f['inhibitor'].map(str) + f['baron'].map(str) + f['dragon'].map(str) \
    #        + f['riftHerald'].map(str) + f['blood&dragon'].map(str) + f['dragon&riftHerald'].map(str) + f['baron&riftHerald'].map(str)\
    #        + f['tower&inhibitor'].map(str) + f['diff_kills'].map(str) + f['diff_assists'].map(str)\
    #        + f['diff_deaths'].map(str) + f['diff_gold'].map(str) + f['diff_KDA'].map(str)
    
    return f

In [5]:
# 读入数据
df = pd.read_csv('exp3.csv')
df.head() 

,index,queueId,mapId,seasonId,team1_win,team1_firstBlood,team1_firstTower,team1_firstInhibitor,team1_firstBaron,team1_firstDragon,...,player9_goldEarned,player9_role,player9_lane,player10_championId,player10_kills,player10_deaths,player10_assists,player10_goldEarned,player10_role,player10_lane
0,0,420,11,13,0,True,False,False,False,False,...,6967,DUO_SUPPORT,NONE,157,3,0,8,8168,DUO,NONE
1,1,420,11,13,1,False,True,True,False,True,...,8133,SOLO,MIDDLE,555,3,7,4,6943,DUO_SUPPORT,BOTTOM
2,2,420,11,13,0,False,True,False,False,True,...,11868,DUO_CARRY,BOTTOM,122,2,2,1,9685,SOLO,TOP
3,3,420,11,13,0,True,False,False,False,True,...,6703,DUO_SUPPORT,BOTTOM,23,6,2,10,13109,SOLO,TOP
4,4,420,11,13,1,True,True,True,False,False,...,6823,DUO,MIDDLE,25,5,7,8,6809,DUO_SUPPORT,BOTTOM


In [6]:
df[df['queueId']==440].index
df =df.drop(labels=df[df['queueId']==440].index,axis=0)

df = diff_KDA(df)
df[df['diff_KDA']>100].index
df =df.drop(labels=df[df['diff_KDA']>100].index,axis=0)
df[df['diff_KDA']<-100].index
df =df.drop(labels=df[df['diff_KDA']<-100].index,axis=0)
df = df.dropna(axis=0,how='any')
df.shape

(75201, 88)

In [7]:
h = int(75201/5)
H = [0,h,2*h,3*h,4*h,75201]
#随机打乱
df=df.reindex(np.random.permutation(df.index))


In [29]:
accuracy = []
for i in range(5):
    test = df[H[i]:H[i+1]]
    train = pd.concat([df[0:H[i]], df[H[i+1]:75201]], axis=0)
    
    train = gettrain(train)
    train_f = getfeature(train,train)
    test_f = getfeature(train,test)
    
    right = 0
    n = test.shape[0]
    k = 37
    
    t0 = np.array(train_f[['sum','win']])
    t1 = np.array(test_f['sum']).tolist()
    t2 = np.array(test_f['win']).tolist()
    
    time_start=time.time()
    for j in range(n):  
        t3 = abs(t0 - [t1[j],0])
        idex=np.lexsort([t3[:,0]])
        temp = pd.DataFrame(t0[idex, :][0:k])[1] #排序取前k个    
        result = temp.value_counts().index[0] #统计取第一个 
        if result == t2[j]:
            right += 1
    accuracy  += [right/n]
    time_end=time.time()
    print('Fold',i+1,':','准确率为：%.3f'%(accuracy[i]),'耗时：%.6f s'%(time_end-time_start))

sum = 0
for i in accuracy:
    sum = sum + i
avg_acc = sum/5
print('平均准确率：%.3f'%(avg_acc))

Fold 1 : 准确率为：0.976 耗时：7.224521 s
Fold 2 : 准确率为：0.983 耗时：7.225293 s
Fold 3 : 准确率为：0.979 耗时：7.323230 s
Fold 4 : 准确率为：0.971 耗时：7.489064 s
Fold 5 : 准确率为：0.982 耗时：7.883754 s
平均准确率：0.978


In [15]:
#重心KNN
accuracy = []
for i in range(5):
    test = df[H[i]:H[i+1]]
    train = pd.concat([df[0:H[i]], df[H[i+1]:75201]], axis=0)
    
    train = gettrain(train)
    train_f = getfeature(train,train)
    test_f = getfeature(train,test)
    
    train_f_1 = train_f[train_f['win']==1]
    train_f_0 = train_f[train_f['win']==0]
    center_1 = train_f_1.sort_values(by = ['sum'])[(train_f_1.shape[0]//2):(train_f_1.shape[0]//2+1)]['sum'].values[0]
    center_0 = train_f_0.sort_values(by = ['sum'])[(train_f_0.shape[0]//2):(train_f_0.shape[0]//2+1)]['sum'].values[0]

    right = 0
    n = test.shape[0]

    t1 = np.array(test_f['sum']).tolist()
    t2 = np.array(test_f['win']).tolist()
    
    time_start=time.time()
    for j in range(n):  
        if abs(center_1-t1[j]) <= abs(center_0-t1[j]):
            result = 1
        else:
            result = 0
        if result == t2[j]:
            right += 1
    accuracy  += [right/n]
    time_end=time.time()
    print('Fold',i+1,':','准确率为：%.3f'%(accuracy[i]),'耗时：%.6f s'%(time_end-time_start))

sum = 0
for i in accuracy:
    sum = sum + i
avg_acc = sum/5
print('平均准确率：%.3f'%(avg_acc))

Fold 1 : 准确率为：0.978 耗时：0.017894 s
Fold 2 : 准确率为：0.978 耗时：0.019946 s
Fold 3 : 准确率为：0.978 耗时：0.019946 s
Fold 4 : 准确率为：0.978 耗时：0.028446 s
Fold 5 : 准确率为：0.977 耗时：0.020291 s
平均准确率：0.978
